# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846690176429                   -0.70    4.5         
  2   -7.852288877771       -2.25       -1.53    1.0   23.7ms
  3   -7.852613942761       -3.49       -2.56    1.5   26.3ms
  4   -7.852646023079       -4.49       -2.88    2.8   32.9ms
  5   -7.852646491170       -6.33       -3.15    1.0   23.6ms
  6   -7.852646678792       -6.73       -4.06    1.0   23.7ms
  7   -7.852646686672       -8.10       -5.14    2.0   29.9ms
  8   -7.852646686727      -10.27       -5.56    1.8   28.0ms
  9   -7.852646686728      -11.88       -5.63    1.5   26.2ms
 10   -7.852646686730      -11.73       -6.68    1.0   24.1ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846781672809                   -0.70           4.8         
  2   -7.852524122506       -2.24       -1.63   0.80    2.0    260ms
  3   -7.852636137718       -3.95       -2.71   0.80    1.0   22.1ms
  4   -7.852646452967       -4.99       -3.29   0.80    2.2   29.1ms
  5   -7.852646682512       -6.64       -4.16   0.80    1.5   25.2ms
  6   -7.852646686391       -8.41       -4.86   0.80    1.2   23.5ms
  7   -7.852646686724       -9.48       -5.78   0.80    2.0   28.0ms
  8   -7.852646686730      -11.24       -6.78   0.80    2.0   27.5ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.397886e+01     3.225698e+00
 * time: 0.5280659198760986
     1     1.626213e+00     1.639029e+00
 * time: 0.7212350368499756
     2    -1.428603e+00     1.828125e+00
 * time: 0.7439520359039307
     3    -3.776991e+00     1.670345e+00
 * time: 0.776879072189331
     4    -5.205702e+00     1.430526e+00
 * time: 0.8098890781402588
     5    -6.846106e+00     8.849961e-01
 * time: 0.8429510593414307
     6    -6.977307e+00     1.056861e+00
 * time: 0.8657059669494629
     7    -7.534875e+00     4.591650e-01
 * time: 0.8884499073028564
     8    -7.641885e+00     3.147881e-01
 * time: 0.9111299514770508
     9    -7.699276e+00     1.030460e-01
 * time: 0.9339380264282227
    10    -7.735008e+00     1.122988e-01
 * time: 0.9568078517913818
    11    -7.763552e+00     1.144315e-01
 * time: 0.9796040058135986
    12    -7.790969e+00     8.544607e-02
 * time: 1.0029919147491455
    13    -7.816070e+00     6.705947e-02
 * time: 1.02580189704

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846832846459                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645908347                   -1.64         
  2   -7.852646686730       -6.11       -3.71    1.60s
  3   -7.852646686730      -13.32       -7.25    147ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.773155770002709e-7
|ρ_newton - ρ_scfv| = 1.305499234323868e-7
|ρ_newton - ρ_dm|   = 1.2979274827882344e-9
